In [ ]:
%load_ext autoreload
%autoreload 2

In [23]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu

import plotly.express as px

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [ ]:
scope = sy.Define_Scope()
w_df_all=uw.build_w_df_all(scope['geo_df'], scope['w_vars'], scope['geo_input_file'], scope['geo_output_column'])
w_df_all.keys()

In [25]:
test_df=w_df_all['hist'][['IL_Prec']]

ref_year_start=dt(2022,11,4)
test_df['seas_day'] = [uw.seas_day(d,ref_year_start) for d in test_df.index]

test_df.head(15)
# test_df.tail(15)

uu.show_excel_index(test_df)


In [ ]:
date_start=dt(2022,5,1)
date_end=dt(2022,5,1)

massive_df = um.get_massive_df(w_df_all['hist'], date_start=date_start, date_end=date_end)
massive_df.shape